A FAIRE :
- Calcul du score d'effort culinaire
- Comparer avec la popularité
- Faire des logs à chaque fois que le dataset est modifié (plutôt que d'en avoir 7 en avoir qu'un que l'on remplace à chaque fois)

In [ ]:
# Charger le nouveau dataset
recipe = pd.read_csv("recipe/7_recipe_n_ingredients_category.csv")
print(recipe.columns)
print(recipe.shape)

In [ ]:
Facile - modéré - Difficile

Exemple de code pour le score d'effort avec les variables et les poids donnés et code suivant pour la création d'un variable qualitative

In [ ]:
# Calcul du Score d'Effort Culinaire
from sklearn.preprocessing import MinMaxScaler

# Normaliser les 4 dimensions sur une échelle 0-1
scaler = MinMaxScaler()
variables_to_scale = ['log_minutes', 'n_steps', 'avg_words_per_step', 'log_n_ingredients']

for var in variables_to_scale:
    recipe[f'{var}_scaled'] = scaler.fit_transform(recipe[[var]])

# Poids pour chaque composante (ajustables)
weights = {
    'time_weight': 0.3,        # 30% temps
    'steps_weight': 0.25,      # 25% étapes 
    'complexity_weight': 0.2,  # 20% complexité instructions
    'ingredients_weight': 0.25 # 25% ingrédients
}

# Calculer le score composite (0-100)
recipe['effort_score'] = (
    weights['time_weight'] * recipe['log_minutes_scaled'] +
    weights['steps_weight'] * recipe['n_steps_scaled'] +
    weights['complexity_weight'] * recipe['avg_words_per_step_scaled'] +
    weights['ingredients_weight'] * recipe['log_n_ingredients_scaled']
) * 100

print("Score d'effort culinaire calculé")
print(f"Score moyen: {recipe['effort_score'].mean():.1f}/100")
print(f"Écart-type: {recipe['effort_score'].std():.1f}")
print(f"Min-Max: {recipe['effort_score'].min():.1f} - {recipe['effort_score'].max():.1f}")
print("="*50)

In [ ]:
# Catégoriser le score d'effort
def categorize_effort(score):
    if score <= 15:
        return 'Très Facile'
    elif score <= 20:
        return 'Facile'
    elif score <= 25:
        return 'Modéré'
    elif score <= 30:
        return 'Difficile'
    else:
        return 'Très Difficile'

recipe['effort_category'] = recipe['effort_score'].apply(categorize_effort)

print("Catégories d'effort créées")
effort_dist = recipe['effort_category'].value_counts()
for category, count in effort_dist.items():
    percentage = (count / len(recipe)) * 100
    print(f"{category}: {count:,} recettes ({percentage:.1f}%)")
print("="*50)